In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [2]:
import numpy as np
train_tensors_original = np.load(open('train_tensors_original.npy'))
test_tensors_original = np.load(open('test_tensors_original.npy'))
valid_tensors_original = np.load(open('valid_tensors_original.npy'))

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

batch_size = 16

# create and configure augmented image generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.2,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.2,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

    
# fit augmented image generator on data
train_generator = train_datagen.flow(train_tensors_original, train_targets, batch_size=batch_size)

In [4]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.2,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.2,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

validation_generator = valid_datagen.flow(valid_tensors_original, valid_targets, batch_size=batch_size)

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

base_model = keras.applications.ResNet50(include_top=False, weights='imagenet')

In [15]:
from keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(133, activation='softmax')(x)
ResNet50_model = Model(inputs = base_model.input, outputs = predictions)

In [16]:
from keras import optimizers
for layer in base_model.layers:
    layer.trainable = False

ResNet50_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ResNet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [18]:
# TRAINING WITH DATA AUGMENTATION (need to recompile the model before running)

from keras.callbacks import ModelCheckpoint  
from keras.callbacks import EarlyStopping

epochs = 40

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best_resnet_augmented.hdf5', 
                               verbose=1, save_best_only=True)

# Using Image Augmentation
ResNet50_model.fit_generator(train_generator,
                    steps_per_epoch=train_tensors_original.shape[0] // batch_size,
                    validation_data=validation_generator, 
                    validation_steps=valid_tensors_original.shape[0] // batch_size,
                    epochs=epochs, 
                    callbacks=[checkpointer, EarlyStopping(min_delta=1e-7, patience=15)], verbose=1)




Epoch 1/40
418/417 [==============================] - 125s 300ms/step - loss: 0.8037 - acc: 0.7662 - val_loss: 1.1358 - val_acc: 0.7174
Epoch 2/40
418/417 [==============================] - 124s 298ms/step - loss: 0.8046 - acc: 0.7696 - val_loss: 1.2524 - val_acc: 0.6719
Epoch 3/40
418/417 [==============================] - 125s 299ms/step - loss: 0.7271 - acc: 0.7823 - val_loss: 1.2191 - val_acc: 0.6946
Epoch 4/40
418/417 [==============================] - 125s 299ms/step - loss: 0.7690 - acc: 0.7833 - val_loss: 1.1948 - val_acc: 0.7102
Epoch 5/40
418/417 [==============================] - 125s 298ms/step - loss: 0.7664 - acc: 0.7847 - val_loss: 1.2403 - val_acc: 0.6994
Epoch 6/40
418/417 [==============================] - 125s 299ms/step - loss: 0.7755 - acc: 0.7799 - val_loss: 1.3201 - val_acc: 0.6982
Epoch 7/40
418/417 [==============================] - 125s 299ms/step - loss: 0.7695 - acc: 0.7871 - val_loss: 1.3532 - val_acc: 0.6754
Epoch 8/40
418/417 [============================

In [54]:
### Load the model weights with the best validation loss.
ResNet50_model.load_weights('saved_models/weights.best_resnet_augmented.hdf5')

In [60]:
from keras.applications.resnet50 import preprocess_input
test_tensors_resnet_preprocessed = preprocess_input(test_tensors_original)
valid_tensors_resnet_preprocessed = preprocess_input(valid_tensors_original)

In [59]:
# get index of predicted dog breed for each image in test set
ResNet50_predictions = [np.argmax(ResNet50_model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors_resnet_preprocessed]

# report test accuracy
test_accuracy = 100*np.sum(np.array(ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 75.0000%


In [61]:
# get index of predicted dog breed for each image in test set
ResNet50_predictions = [np.argmax(ResNet50_model.predict(np.expand_dims(feature, axis=0))) for feature in valid_tensors_resnet_preprocessed]

# report test accuracy
valid_accuracy = 100.0*np.sum(np.array(ResNet50_predictions)==np.argmax(valid_targets, axis=1))/len(ResNet50_predictions)
print('Test accuracy: %.4f%%' % valid_accuracy)

Test accuracy: 77.2455%


In [ ]:
#INCEPTION

In [21]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

batch_size = 16

# create and configure augmented image generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.2,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.2,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

    
# fit augmented image generator on data
train_generator = train_datagen.flow(train_tensors_original, train_targets, batch_size=batch_size)

In [22]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.2,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.2,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

validation_generator = valid_datagen.flow(valid_tensors_original, valid_targets, batch_size=batch_size)

In [23]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

base_model = keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [24]:
from keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(133, activation='softmax')(x)
inception_model = Model(inputs = base_model.input, outputs = predictions)

In [25]:
from keras import optimizers
for layer in base_model.layers:
    layer.trainable = False

inception_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

inception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_50 (Activation)      (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [26]:
# TRAINING WITH DATA AUGMENTATION (need to recompile the model before running)

from keras.callbacks import ModelCheckpoint  
from keras.callbacks import EarlyStopping

epochs = 50

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best_inception_augmented.hdf5', 
                               verbose=1, save_best_only=True)

# Using Image Augmentation
inception_model.fit_generator(train_generator,
                    steps_per_epoch=train_tensors_original.shape[0] // batch_size,
                    validation_data=validation_generator, 
                    validation_steps=valid_tensors_original.shape[0] // batch_size,
                    epochs=epochs, 
                    callbacks=[checkpointer, EarlyStopping(min_delta=1e-7, patience=15)], verbose=1)




Epoch 1/50
418/417 [==============================] - 119s 284ms/step - loss: 3.3745 - acc: 0.2637 - val_loss: 1.9733 - val_acc: 0.5078
Epoch 2/50
418/417 [==============================] - 103s 247ms/step - loss: 2.1896 - acc: 0.4445 - val_loss: 1.7804 - val_acc: 0.5365
Epoch 3/50
418/417 [==============================] - 103s 247ms/step - loss: 1.9694 - acc: 0.4979 - val_loss: 1.7322 - val_acc: 0.5485
Epoch 4/50
418/417 [==============================] - 103s 247ms/step - loss: 1.8517 - acc: 0.5195 - val_loss: 1.6753 - val_acc: 0.5725
Epoch 5/50
418/417 [==============================] - 103s 247ms/step - loss: 1.7734 - acc: 0.5376 - val_loss: 1.6443 - val_acc: 0.5784
Epoch 6/50
418/417 [==============================] - 103s 246ms/step - loss: 1.7573 - acc: 0.5461 - val_loss: 1.6614 - val_acc: 0.5713
Epoch 7/50
418/417 [==============================] - 103s 246ms/step - loss: 1.7113 - acc: 0.5533 - val_loss: 1.6525 - val_acc: 0.5892
Epoch 8/50
418/417 [============================

Epoch 31/50
418/417 [==============================] - 103s 246ms/step - loss: 1.6350 - acc: 0.6013 - val_loss: 1.7212 - val_acc: 0.6216


In [27]:
### Load the model weights with the best validation loss.
inception_model.load_weights('saved_models/weights.best_inception_augmented.hdf5')

In [63]:
from keras.applications.inception_v3 import preprocess_input
test_tensors_inception_preprocessed = preprocess_input(test_tensors_original)
valid_tensors_inception_preprocessed = preprocess_input(valid_tensors_original)

In [68]:
# get index of predicted dog breed for each image in test set
inception_predictions = [np.argmax(inception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors_inception_preprocessed]

# report test accuracy
test_accuracy = 100.0*np.sum(np.array(inception_predictions)==np.argmax(test_targets, axis=1))/len(inception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 0.0060%


In [66]:
# get index of predicted dog breed for each image in test set
inception_predictions = [np.argmax(inception_model.predict(np.expand_dims(feature, axis=0))) for feature in valid_tensors_inception_preprocessed]

# report test accuracy
test_accuracy = 100*np.sum(np.array(inception_predictions)==np.argmax(valid_targets, axis=1))/len(inception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 3.0000%


In [ ]:
#xception

In [29]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input

batch_size = 16

# create and configure augmented image generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

    
# fit augmented image generator on data
train_generator = train_datagen.flow(train_tensors_original, train_targets, batch_size=batch_size)

In [30]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

validation_generator = valid_datagen.flow(valid_tensors_original, valid_targets, batch_size=batch_size)

In [31]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

base_model = keras.applications.Xception(include_top=False, weights='imagenet')

In [32]:
from keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(133, activation='softmax')(x)
xception_model = Model(inputs = base_model.input, outputs = predictions)

In [33]:
from keras import optimizers
for layer in base_model.layers:
    layer.trainable = False

xception_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

xception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [34]:
# TRAINING WITH DATA AUGMENTATION (need to recompile the model before running)

from keras.callbacks import ModelCheckpoint  
from keras.callbacks import EarlyStopping

epochs = 50

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best_xception_augmented.hdf5', 
                               verbose=1, save_best_only=True)

# Using Image Augmentation
xception_model.fit_generator(train_generator,
                    steps_per_epoch=train_tensors_original.shape[0] // batch_size,
                    validation_data=validation_generator, 
                    validation_steps=valid_tensors_original.shape[0] // batch_size,
                    epochs=epochs, 
                    callbacks=[checkpointer, EarlyStopping(min_delta=1e-7, patience=15)], verbose=1)




Epoch 1/50
418/417 [==============================] - 152s 363ms/step - loss: 2.8360 - acc: 0.4313 - val_loss: 1.4338 - val_acc: 0.6515
Epoch 2/50
418/417 [==============================] - 143s 342ms/step - loss: 1.5125 - acc: 0.6521 - val_loss: 1.2511 - val_acc: 0.6910
Epoch 3/50
418/417 [==============================] - 143s 342ms/step - loss: 1.2952 - acc: 0.6790 - val_loss: 1.1435 - val_acc: 0.7066
Epoch 4/50
418/417 [==============================] - 143s 342ms/step - loss: 1.1408 - acc: 0.7076 - val_loss: 1.0448 - val_acc: 0.7150
Epoch 5/50
418/417 [==============================] - 143s 342ms/step - loss: 1.0716 - acc: 0.7193 - val_loss: 1.0554 - val_acc: 0.7246
Epoch 6/50
418/417 [==============================] - 143s 342ms/step - loss: 0.9979 - acc: 0.7365 - val_loss: 1.0569 - val_acc: 0.7126
Epoch 7/50
418/417 [==============================] - 143s 342ms/step - loss: 0.9580 - acc: 0.7424 - val_loss: 1.0217 - val_acc: 0.7317
Epoch 8/50
418/417 [============================

In [36]:
### Load the model weights with the best validation loss.
xception_model.load_weights('saved_models/weights.best_xception_augmented.hdf5')

In [37]:
# get index of predicted dog breed for each image in test set
xception_predictions = [np.argmax(xception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors_original]

# report test accuracy
test_accuracy = 100.0*np.sum(np.array(xception_predictions)==np.argmax(test_targets, axis=1))/len(xception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 0.0000%


In [ ]:
#vgg16

In [38]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

batch_size = 16

# create and configure augmented image generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

    
# fit augmented image generator on data
train_generator = train_datagen.flow(train_tensors_original, train_targets, batch_size=batch_size)

In [39]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

validation_generator = valid_datagen.flow(valid_tensors_original, valid_targets, batch_size=batch_size)

In [40]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

base_model = keras.applications.VGG16(include_top=False, weights='imagenet')

In [41]:
from keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(2 * 133,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2 * 133,activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(133, activation='softmax')(x)
vgg16_model = Model(inputs = base_model.input, outputs = predictions)

In [42]:
from keras import optimizers
for layer in base_model.layers:
    layer.trainable = False

vgg16_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [43]:
# TRAINING WITH DATA AUGMENTATION (need to recompile the model before running)

from keras.callbacks import ModelCheckpoint  
from keras.callbacks import EarlyStopping

epochs = 50

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best_vgg16_augmented.hdf5', 
                               verbose=1, save_best_only=True)

# Using Image Augmentation
vgg16_model.fit_generator(train_generator,
                    steps_per_epoch=train_tensors_original.shape[0] // batch_size,
                    validation_data=validation_generator, 
                    validation_steps=valid_tensors_original.shape[0] // batch_size,
                    epochs=epochs, 
                    callbacks=[checkpointer, EarlyStopping(min_delta=1e-7, patience=15)], verbose=1)




Epoch 1/50
418/417 [==============================] - 153s 365ms/step - loss: 8.6589 - acc: 0.0112 - val_loss: 4.8528 - val_acc: 0.0156
Epoch 2/50
418/417 [==============================] - 146s 348ms/step - loss: 4.8883 - acc: 0.0235 - val_loss: 4.7536 - val_acc: 0.0527
Epoch 3/50
418/417 [==============================] - 145s 348ms/step - loss: 4.7699 - acc: 0.0302 - val_loss: 4.5897 - val_acc: 0.0719
Epoch 4/50
418/417 [==============================] - 145s 348ms/step - loss: 4.6469 - acc: 0.0429 - val_loss: 4.4210 - val_acc: 0.1138
Epoch 5/50
418/417 [==============================] - 145s 348ms/step - loss: 4.4957 - acc: 0.0546 - val_loss: 4.1458 - val_acc: 0.1749
Epoch 6/50
418/417 [==============================] - 146s 348ms/step - loss: 4.3705 - acc: 0.0757 - val_loss: 3.9497 - val_acc: 0.1772
Epoch 7/50
418/417 [==============================] - 145s 348ms/step - loss: 4.2260 - acc: 0.0814 - val_loss: 3.6672 - val_acc: 0.2299
Epoch 8/50
418/417 [============================

Epoch 27/50
418/417 [==============================] - 146s 348ms/step - loss: 3.2721 - acc: 0.1845 - val_loss: 2.5030 - val_acc: 0.4096
Epoch 28/50
418/417 [==============================] - 145s 348ms/step - loss: 3.2491 - acc: 0.1897 - val_loss: 2.5042 - val_acc: 0.4287
Epoch 29/50
418/417 [==============================] - 146s 348ms/step - loss: 3.2796 - acc: 0.1830 - val_loss: 2.4620 - val_acc: 0.4287
Epoch 30/50
418/417 [==============================] - 145s 348ms/step - loss: 3.2223 - acc: 0.1954 - val_loss: 2.4953 - val_acc: 0.3856
Epoch 31/50
418/417 [==============================] - 145s 348ms/step - loss: 3.2046 - acc: 0.1912 - val_loss: 2.4526 - val_acc: 0.4228
Epoch 32/50
418/417 [==============================] - 145s 348ms/step - loss: 3.2257 - acc: 0.1935 - val_loss: 2.4187 - val_acc: 0.3760
Epoch 33/50
418/417 [==============================] - 145s 348ms/step - loss: 3.2296 - acc: 0.1926 - val_loss: 2.4130 - val_acc: 0.4251
Epoch 34/50
418/417 [====================

In [44]:
### Load the model weights with the best validation loss.
vgg16_model.load_weights('saved_models/weights.best_vgg16_augmented.hdf5')

In [69]:
from keras.applications.vgg16 import preprocess_input
test_tensors_vgg16_preprocessed = preprocess_input(test_tensors_original)
valid_tensors_vgg16_preprocessed = preprocess_input(valid_tensors_original)

In [70]:
# get index of predicted dog breed for each image in test set
vgg16_predictions = [np.argmax(vgg16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors_vgg16_preprocessed]

# report test accuracy
test_accuracy = 100*np.sum(np.array(vgg16_predictions)==np.argmax(test_targets, axis=1))/len(vgg16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 0.0000%


In [ ]:
#vgg19

In [46]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import preprocess_input

batch_size = 19

# create and configure augmented image generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

    
# fit augmented image generator on data
train_generator = train_datagen.flow(train_tensors_original, train_targets, batch_size=batch_size)

In [47]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2) # randomly zooming inside pictures

validation_generator = valid_datagen.flow(valid_tensors_original, valid_targets, batch_size=batch_size)

In [48]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

base_model = keras.applications.VGG19(include_top=False, weights='imagenet')

80150528/80134624 [==============================] - 5s 0us/step


In [49]:
from keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(2 * 133,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2 * 133,activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(133, activation='softmax')(x)
vgg19_model = Model(inputs = base_model.input, outputs = predictions)

In [50]:
from keras import optimizers
for layer in base_model.layers:
    layer.trainable = False

vgg19_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

vgg19_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [51]:
# TRAINING WITH DATA AUGMENTATION (need to recompile the model before running)

from keras.callbacks import ModelCheckpoint  
from keras.callbacks import EarlyStopping

epochs = 50

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best_vgg19_augmented.hdf5', 
                               verbose=1, save_best_only=True)

# Using Image Augmentation
vgg19_model.fit_generator(train_generator,
                    steps_per_epoch=train_tensors_original.shape[0] // batch_size,
                    validation_data=validation_generator, 
                    validation_steps=valid_tensors_original.shape[0] // batch_size,
                    epochs=epochs, 
                    callbacks=[checkpointer, EarlyStopping(min_delta=1e-7, patience=15)], verbose=1)




Epoch 1/50
352/351 [==============================] - 219s 623ms/step - loss: 8.1026 - acc: 0.0129 - val_loss: 4.8660 - val_acc: 0.0240
Epoch 2/50
352/351 [==============================] - 213s 604ms/step - loss: 4.8835 - acc: 0.0227 - val_loss: 4.7976 - val_acc: 0.0587
Epoch 3/50
352/351 [==============================] - 213s 604ms/step - loss: 4.7738 - acc: 0.0355 - val_loss: 4.5983 - val_acc: 0.0790
Epoch 4/50
352/351 [==============================] - 213s 604ms/step - loss: 4.6194 - acc: 0.0530 - val_loss: 4.3332 - val_acc: 0.1281
Epoch 5/50
352/351 [==============================] - 213s 605ms/step - loss: 4.4986 - acc: 0.0615 - val_loss: 4.1205 - val_acc: 0.1581
Epoch 6/50
352/351 [==============================] - 213s 605ms/step - loss: 4.3282 - acc: 0.0766 - val_loss: 3.8517 - val_acc: 0.2024
Epoch 7/50
352/351 [==============================] - 213s 604ms/step - loss: 4.2473 - acc: 0.0873 - val_loss: 3.6790 - val_acc: 0.2443
Epoch 8/50
352/351 [============================

352/351 [==============================] - 213s 604ms/step - loss: 3.1374 - acc: 0.2133 - val_loss: 2.2930 - val_acc: 0.4443
Epoch 27/50
352/351 [==============================] - 213s 604ms/step - loss: 3.1156 - acc: 0.2124 - val_loss: 2.2815 - val_acc: 0.4575
Epoch 28/50
352/351 [==============================] - 213s 604ms/step - loss: 3.0898 - acc: 0.2171 - val_loss: 2.2679 - val_acc: 0.4599
Epoch 29/50
352/351 [==============================] - 213s 604ms/step - loss: 3.0914 - acc: 0.2231 - val_loss: 2.2783 - val_acc: 0.4539
Epoch 30/50
352/351 [==============================] - 213s 604ms/step - loss: 3.0969 - acc: 0.2146 - val_loss: 2.2290 - val_acc: 0.4754
Epoch 31/50
352/351 [==============================] - 213s 604ms/step - loss: 3.0832 - acc: 0.2216 - val_loss: 2.2231 - val_acc: 0.4611
Epoch 32/50
352/351 [==============================] - 213s 604ms/step - loss: 3.0620 - acc: 0.2235 - val_loss: 2.2226 - val_acc: 0.4623
Epoch 33/50
352/351 [==============================] 

In [52]:
### Load the model weights with the best validation loss.
vgg19_model.load_weights('saved_models/weights.best_vgg19_augmented.hdf5')

In [72]:
from keras.applications.vgg19 import preprocess_input
test_tensors_vgg19_preprocessed = preprocess_input(test_tensors_original)
valid_tensors_vgg19_preprocessed = preprocess_input(valid_tensors_original)

In [73]:
# get index of predicted dog breed for each image in test set
vgg19_predictions = [np.argmax(vgg19_model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors_vgg19_preprocessed]

# report test accuracy
test_accuracy = 100*np.sum(np.array(vgg19_predictions)==np.argmax(test_targets, axis=1))/len(vgg19_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 1.0000%
